In [11]:
from dotenv import load_dotenv
from pinecone import Pinecone
import os
import google.generativeai as genai

load_dotenv()

api_key = os.getenv("PINE_CONE_KEY")
pc = Pinecone(api_key = api_key)
index = pc.Index("f1rag")

api_key = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=api_key)

In [14]:
user_query = "How long can M1 students stay in USA on Visa" 

query_vector = genai.embed_content(
    model="models/embedding-001",   # same model as you used before
    content=user_query
)["embedding"]

In [ ]:
results = index.query(
    vector=query_vector,
    top_k = 5,                # number of most relevant chunks to fetch
    include_metadata=True,  # so you also get back your original text
    namespace="f1rag"
)

In [16]:
print(results)

{'matches': [{'id': '13_chunk_12',
              'metadata': {'chunk_index': 12.0,
                           'docid': 13.0,
                           'notes': 'This document outlines the regulations '
                                    'regarding Change of Status (COS), '
                                    'Extension of Stay, and Length of Stay for '
                                    'F-1 (academic) and M-1 (vocational) '
                                    'nonimmigrant students in the United '
                                    'States.',
                           'text': '*   **M-1 Students:** Admitted for a fixed '
                                   'period, which is the time needed to '
                                   'complete the course, plus any authorized '
                                   'practical training, plus 30 days for '
                                   'departure. The total period cannot exceed '
                                   '1 year. They can be a

In [17]:
text = ""
for match in results["matches"]:
    score = match["score"]             # similarity score
    text += match["metadata"]["text"]   # your stored chunk
    doc_id = match["metadata"]["docid"]   # original doc ID
    
    print(f"Score: {score:.4f} | Doc: {doc_id}")
    print(f"Chunk: {text[:200]}...\n")

Score: 0.8660 | Doc: 13.0
Chunk: *   **M-1 Students:** Admitted for a fixed period, which is the time needed to complete the course, plus any authorized practical training, plus 30 days for departure. The total period cannot exceed 1...

Score: 0.8240 | Doc: 13.0
Chunk: *   **M-1 Students:** Admitted for a fixed period, which is the time needed to complete the course, plus any authorized practical training, plus 30 days for departure. The total period cannot exceed 1...

Score: 0.8232 | Doc: 13.0
Chunk: *   **M-1 Students:** Admitted for a fixed period, which is the time needed to complete the course, plus any authorized practical training, plus 30 days for departure. The total period cannot exceed 1...

Score: 0.8186 | Doc: 13.0
Chunk: *   **M-1 Students:** Admitted for a fixed period, which is the time needed to complete the course, plus any authorized practical training, plus 30 days for departure. The total period cannot exceed 1...

Score: 0.8176 | Doc: 12.0
Chunk: *   **M-1 Stude

In [18]:
print(text)

*   **M-1 Students:** Admitted for a fixed period, which is the time needed to complete the course, plus any authorized practical training, plus 30 days for departure. The total period cannot exceed 1 year. They can be admitted up to 30 days before the I-20 start date.*   **M-1 Students:** Cumulative time of extensions, including practical training, cannot exceed 3 years from the original start date.
    *   **Application:** M-1 students must apply to USCIS for an extension using Form I-539 at least 15 days, but not more than 60 days, before the I-20 end date. They must demonstrate compelling educational or medical reasons for the delay and intent to maintain status.*   **Period of Stay:** Extensions are granted for the period needed to complete the course plus 30 days to depart, or for a total of 1 year, whichever is less.
    *   **Reinstatement:** Similar to F-1, M-1 students can request reinstatement via Form I-539, meeting conditions such as filing within 5 months, no willful viol

In [19]:
print(len(text))

1767


In [20]:
system_prompt = f""" You are a helpful agent for international students use the following information to provide the answer better
context: {text}
"""

In [21]:
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=system_prompt,
)

In [22]:
response = model.generate_content(user_query)
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "As an M-1 vocational student, your stay in the U.S. has specific time limits:\n\n1.  **Initial Admission:**\n    *   You are admitted for the time needed to complete your course, plus any authorized practical training, plus an additional 30 days to prepare for departure.\n    *   **The total period for this initial admission cannot exceed 1 year.**\n    *   You are also permitted to be admitted into the U.S. up to **30 days before** your I-20 start date.\n\n2.  **Extensions:**\n    *   You may apply for an extension of stay with USCIS using Form I-539 if you need more time due to compelling educational or medical reasons.\n    *   Each extension is granted for the period needed to complete your course plus 30 days to depart, or for a total of 1 year, whichev

In [23]:
response.text

'As an M-1 vocational student, your stay in the U.S. has specific time limits:\n\n1.  **Initial Admission:**\n    *   You are admitted for the time needed to complete your course, plus any authorized practical training, plus an additional 30 days to prepare for departure.\n    *   **The total period for this initial admission cannot exceed 1 year.**\n    *   You are also permitted to be admitted into the U.S. up to **30 days before** your I-20 start date.\n\n2.  **Extensions:**\n    *   You may apply for an extension of stay with USCIS using Form I-539 if you need more time due to compelling educational or medical reasons.\n    *   Each extension is granted for the period needed to complete your course plus 30 days to depart, or for a total of 1 year, whichever is less.\n    *   **Important Cumulative Limit:** The **cumulative time** you can stay in M-1 status, including your initial admission and all extensions (and practical training), **cannot exceed a total of 3 years from your ori